# 2_countrymaps_annotation

### Summary
The resulting list of geometry group is visualized in a ipyleaflet map, which allows to label the groups. The list is continuously saved as a pickle object, therefore saving any changes made to it.

In [ ]:
# Hidden depedency of geopandas: descartes
import pickle
from ipyleaflet import *
import ipywidgets as widgets

# III) Annotation of country parts
The following two cells intiialize an ipyleaflet map, that interactively lets you annotate the country parts. use the following controls:
- Zoom via slider OR **mouse wheel**.
- Click the **Next** and **Previous** buttons to jump from country part to country part.
- The **Country** textfield shows you to what country the current country prt belongt to. It is not editable.
- Enter a fitting name (e.g. Alaska, Azores) for the country part into the textfield **Name** in the upper right.
- **Note:** Each time you navigate from one place to another, your changes are automatically saved in memory as well as a pickle object of the list.

In [ ]:
skip_single_geometry_countries = False

In [ ]:
import pickle
country_list = pickle.load(open('country_list.p', 'rb'))

In [ ]:
m = Map(center=(0,0), zoom=3, scroll_wheel_zoom=True, zoom_control=False, basemap=basemaps.Esri.WorldStreetMap)
marker = Marker(location=(0,0), draggable=False)
m.add_layer(marker)

polygon = Polygon(
    locations=[(0,0)],
    color="green",
    fill_color="green"
)
m.add_layer(polygon)

zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=3)
widgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
widget_control_zoom = WidgetControl(widget=zoom_slider, position='topleft')
m.add_control(widget_control_zoom)

country_text = widgets.Text(
    value='',
    placeholder='',
    description='Country:',
    disabled=True,
    layout={'width': '500px'}
)

text_input = widgets.Text(
    value='',
    placeholder='',
    description='Name:',
    disabled=False,
    layout={'width': '500px'}
)

goto_next = widgets.Button(
    description='Next',
    disabled=False,
    button_style='',
    tooltip='Next geometry',
    icon=''
)

goto_pevious = widgets.Button(
    description='Previous',
    disabled=False,
    button_style='',
    tooltip='Prevous geometry',
    icon=''
)

In [ ]:
index = 0

def goto_index(index):
    global polygon
    
    text_input.value = country_list[index]['name']
    country_text.value = country_list[index]['country']
    centroid = country_list[index]['geometry'].centroid.iloc[0]
    center = (centroid.y, centroid.x)
    
    locations = []
    if country_list[index]['geometry'].iloc[0].geom_type == 'Polygon':
        poly = country_list[index]['geometry'].iloc[0]
        locations = [(y,x) for y,x in zip(poly.exterior.coords.xy[1], poly.exterior.coords.xy[0])]
    if country_list[index]['geometry'].iloc[0].geom_type == 'MultiPolygon':
        for poly in country_list[index]['geometry'].iloc[0]:
            locations.append([(y,x) for y,x in zip(poly.exterior.coords.xy[1], poly.exterior.coords.xy[0])])
    m.remove_layer(polygon)
    polygon = Polygon(
        locations=locations,
        color="green",
        fill_color="green"
    )
    m.add_layer(polygon)
    m.center = center
    marker.location = center
    
def next_clicked(button):
    global index
    country_list[index]['name'] = text_input.value
    pickle.dump(country_list, open('country_list.p', 'wb'))
    while True:
        index += 1
        if index == len(country_list):
            index = 0
        if country_list[index]['nr_parts'] > 1 or not skip_single_geometry_countries:
            break
    goto_index(index)
    
def previous_clicked(button):
    global index
    country_list[index]['name'] = text_input.value
    pickle.dump(country_list, open('country_list.p', 'wb'))
    while True:
        index -= 1
        if index < 0:
            index = len(country_list) - 1
        if country_list[index]['nr_parts'] > 1 or not skip_single_geometry_countries:
            break
    goto_index(index)
    
goto_next.on_click(next_clicked)
goto_pevious.on_click(previous_clicked)

widget_control_country = WidgetControl(widget=country_text, position='topright')
m.add_control(widget_control_country)
widget_control_input = WidgetControl(widget=text_input, position='topright')
m.add_control(widget_control_input)
widget_control_next = WidgetControl(widget=goto_next, position='bottomright')
m.add_control(widget_control_next)
widget_control_previous = WidgetControl(widget=goto_pevious, position='bottomleft')
m.add_control(widget_control_previous)

next_clicked(None)
m

Running the cell above should show a map. If this is not the case, try to reboot your jupyter notebook.

# IV) Data preparation for layout

In [ ]:
country_dict = {}
for c in country_list:
    adm0_a3 = c['ADM0_A3']
    if adm0_a3 not in country_dict:
        country_dict[adm0_a3] = {
            'name': c['name'],
            'ADM0_A3': c['ADM0_A3'],
            'nr_parts': c['nr_parts'],
            'rows': 1,
            'cols': 1,
            'parts': [],
        }
    colroot = len(country_dict[adm0_a3]['parts'])
    country_dict[adm0_a3]['cols'] = colroot + 1
    country_dict[adm0_a3]['parts'].append({
        'name': c['name'],
        'main': c['main'],
        'geometry': c['geometry'],
        'rowroot': 0,
        'colroot': colroot,
        'rowspan': 1,
        'colspan': 1,
    })
pickle.dump(country_dict, open('country_dict.p', 'wb'))
country_list_layout = [v for v in country_dict.values()]
pickle.dump(country_list_layout, open('country_list_layout.p', 'wb'))